In [ ]:
""" ARBOL CON multiclase BINARIA BENIGNOS Y MALWARE"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix 
from sklearn import datasets, metrics
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report 
import os
import glob
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
import seaborn as sns

"""crea entorno grÃ¡fico"""
plt.rcParams['figure.figsize'] = (40, 50)
plt.style.use('ggplot')

# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
os.chdir("..\\Datasets")
os.getcwd()
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

#Para chequear que todo está bien, mostramos la list_data por consola
#list_data
 
df = pd.concat(list_data,ignore_index=True)

tipos=df.dtypes  #tipos de datos de campos

"""información datos"""
print(df.head(5))
print('Cantidad de Filas y columnas:',df.shape)
print('Nombre columnas:',df.columns)

"""Columnas, nulos y tipo de datos"""
print(df.info())

"""descripción estadística de los datos numéricos"""
print(df.describe())

"""correlación entre los datos"""
corr = df.set_index('Label').corr()
#sm.graphics.plot_corr(corr, xnames=list(corr.columns))
#plt.show()

df2=df[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
df3=df[['Label']]

"""convertir columna tipo dato str a int"""
df2['Label']=df3[['Label']]


"""
from sklearn.preprocessing import KBinsDiscretizer
bines = 4 # Elegir el número de bines
cabecera = list(df2) # Guardamos los nombres de las columnas.
ind = 0 # Contador para iterar por columnas.
while (ind < len(cabecera)):
    disc = df2.iloc[:,ind] 
    disc = disc.to_frame() 
    disc = KBinsDiscretizer(n_bins=bines, encode='ordinal',
                            strategy = "quantile").fit_transform(disc)
    df2[cabecera[ind]] = disc 
    ind = ind + 1
    del(disc)   
"""



"""desordena el dataset"""
"""desordena el dataset"""

df2=df2.sort_values('Fwd_Pkt_Len_Min')
df2 = df2[df2['Label'].isin([1,2,3,4])]



""" convertir valores número la clase
df2['Label'] = df2['Label'].replace(1,"Benigno")
df2['Label'] = df2['Label'].replace(2,"Adware")
df2['Label'] = df2['Label'].replace(3,"Scareware")
df2['Label'] = df2['Label'].replace(4,"Ransomware")
#print(df2.head(10))
#print(df2.groupby('Label').size())
"""

dff=df2[:]
dff1 = dff[dff.Label == 4]
dff = dff.drop(dff[dff['Label'] == 4].index)
a = dff1.iloc[0:41100,]
dff2 = pd.concat([dff,a])

dff=dff2[:]
dff1 = dff[dff.Label == 1]
dff = dff.drop(dff[dff['Label'] == 1].index)
a = dff1.iloc[0:123500,]
dff2 = pd.concat([dff,a])

print(dff2.groupby('Label').size())

"""CREAMOS EL MODELO Y LO PROBAMOS CON LOS MISMOS DATOS SIN CROSS"""


"""
def clean_dataset(df):
       assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
       df.dropna(inplace=True)
       indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
       return df[indices_to_keep].astype(np.float64)
clean_dataset(dff2)   
"""
dff2 = dff2.fillna(lambda x: x.median())


# example of a standardization

#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#scaler = StandardScaler()
#dff2 = scaler.fit_transform(dff2)
#trans = MinMaxScaler()
#dff2 = trans.fit_transform(dff2)


   
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, Y_train, Y_test = train_test_split(dff2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 
	   ]], dff2['Label'], test_size = 0.20, random_state=0)



clf2 = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       random_state=0, splitter='best')#presort=False,



#entreno modelo

clf2.fit(X_train, Y_train)

acc_decision_tree = round(clf2.score(X_train, Y_train) * 100, 2)
print("Precisión Modelo entrenado (Accuracy) : ", acc_decision_tree, "%")
acc_decision_tree = round(clf2.score(X_test, Y_test) * 100, 2)
print("Precisión Modelo validado (Accuracy) : ", acc_decision_tree, "%")

#Realizo una predicción
y_pred2 = clf2.predict(X_test)

matriz = confusion_matrix(Y_test, y_pred2)
print('Matriz de Confusión AD Validada :')
print(matriz)

print("Métricas AD Validadas")
print(classification_report(Y_test, y_pred2))



#classifier=OneVsRestClassifier(LinearSVC(random_state=0))
#y_score = classifier.fit(X_train, Y_train).decision_function (X_test)
classifier=OneVsRestClassifier(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       random_state=0, splitter='best'))
y_score = classifier.fit(X_train, Y_train).predict_proba(X_test)



def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
    y_score = clf.predict_proba(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

plot_multiclass_roc(classifier, X_test, Y_test, n_classes=4, figsize=(16, 10))


"""============================================="""
""" CROSS VALIDATION"""

X_trainCV, X_testCV, Y_trainCV, Y_testCV = train_test_split(dff2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 
	   ]],dff2['Label'], test_size=0.2, random_state=6) 

clfCV = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       random_state=0, splitter='best')
clfCV.fit(X_train, Y_train)

kf = KFold(n_splits=10)
score = clfCV.score(X_trainCV,Y_trainCV)
print("Metrica del modelo", score * 100)
scores = cross_val_score(clfCV, X_trainCV, Y_trainCV, cv=kf, scoring="accuracy")
print("Metricas cross_validation", scores * 100)
#print("Media de cross_validation", scores.mean() * 100)
preds = clfCV.predict(X_testCV)
score_pred = metrics.accuracy_score(Y_testCV, preds)
print("Metrica en Test", score_pred * 100)
conf_mat = confusion_matrix(Y_testCV, preds)
print(conf_mat)
print(classification_report(Y_testCV, preds))




#classifierCV=OneVsRestClassifier(LinearSVC(random_state=0))
#y_score = classifierCV.fit(X_train, Y_train).decision_function(X_test)

classifierCV=OneVsRestClassifier(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       random_state=0, splitter='best'))
y_score = classifierCV.fit(X_train, Y_train).predict_proba(X_test)


def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
    y_score = clf.predict_proba(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

#plot_multiclass_roc(classifierCV, X_test, Y_test, n_classes=4, figsize=(16, 10))



"""
dtype: int64
Precisión Modelo entrenado (Accuracy) :  93.48 %
Precisión Modelo validado (Accuracy) :  93.45 %
Matriz de Confusión AD Validada :
[[26036  1556]
 [ 1233 13734]]
Métricas AD Validadas
              precision    recall  f1-score   support
     Benigno       0.95      0.94      0.95     27592
  Ransomware       0.90      0.92      0.91     14967
    accuracy                           0.93     42559
   macro avg       0.93      0.93      0.93     42559
weighted avg       0.93      0.93      0.93     42559
con
dtype: int64
Label
Benigno       137907
Ransomware     74888
dtype: int64
Precisión Modelo entrenado (Accuracy) :  90.92 %
Precisión Modelo validado (Accuracy) :  91.02 %
Matriz de Confusión AD Validada :
[[25505  1958]
 [ 1865 13231]]
Métricas AD Validadas
              precision    recall  f1-score   support
     Benigno       0.93      0.93      0.93     27463
  Ransomware       0.87      0.88      0.87     15096
    accuracy                           0.91     42559
   macro avg       0.90      0.90      0.90     42559
weighted avg       0.91      0.91      0.91     42559
sin
[8 rows x 16 columns]
Label
Benigno    137907
Malware    155426
dtype: int64
Precisión Modelo entrenado (Accuracy) :  92.97 %
Precisión Modelo validado (Accuracy) :  92.84 %
Matriz de Confusión AD Validada :
[[25765  2027]
 [ 2171 28704]]
Métricas AD Validadas
              precision    recall  f1-score   support
     Benigno       0.92      0.93      0.92     27792
     Malware       0.93      0.93      0.93     30875
    accuracy                           0.93     58667
   macro avg       0.93      0.93      0.93     58667
weighted avg       0.93      0.93      0.93     58667
"""

['benign-2c .csv', 'malware.csv']
   Flow_Duration  Tot_Fwd_Pkts  Tot_Bwd_Pkts  TotLen_Fwd_Pkts  \
0           5870             2             0              813   
1           1499             2             0              128   
2             16             0             2                0   
3          46435             1             1               79   
4              0             1             0              356   

   TotLen_Bwd_Pkts  Fwd_Pkt_Len_Min  Bwd_Pkt_Len_Min  Fwd_Pkt_Len_Max  \
0                0               83             -1.0              730   
1                0               64             -1.0               64   
2              159               -1             52.0               -1   
3               95               79             95.0               79   
4                0              356             -1.0              356   

   Bwd_Pkt_Len_Max  Fwd_Pkt_Len_Mean  Bwd_Pkt_Len_Mean  Fwd_Pkt_Len_Std  \
0               -1             406.5               0.0       